In [ ]:
#import packages

import numpy as np
import matplotlib.pyplot as plt
import sys
import scipy.stats as stats
from datetime import datetime
import os
from scipy.optimize import fmin
import pandas as pd
import math as math

## Slow version

In [ ]:
## Actor Critic algorithm

#Algorithm for N=2

class ActorCritic:
    def __init__(self,d,N,v,tc):
        self.mu_hat=np.zeros(d)
        self.y=np.zeros(d)
        self.B=np.eye(d)
        self.theta=np.zeros(d)
        self.v=v
        self.N=N
        self.d=d
        self.Bss=[]
        self.t=0
        self.tc=tc
        self.acs=[]
        self.rwds=[]
        #self.R=np.zeros((d,d))
        self.best=np.zeros(d)
    def select_ac(self,Bs):
        self.t=self.t+1
        self.Bs=Bs
        self.Bss.append(self.Bs)
        pi=np.array([np.exp(np.dot(bi,self.theta)) for bi in Bs])
        #if np.any(np.isinf(pi)):
        #    pi=list(np.isinf(pi)/np.sum(np.isinf(pi)))
        #else:
        pi=list(pi/np.sum(pi))
        #print(pi)
        self.action=np.random.choice(np.arange(self.N),1,p=pi)[0]
        #print(self.action)
        self.acs.append(self.action)
        
        #for bi in Bs:
        #    self.R+=np.outer(bi,bi)
        
        return(self.action)
    def regularized_rwd(self,theta):
        avg_rwd=0
        for tau in range(self.t):
            Bs=self.Bss[tau]
            rhat=np.dot(Bs,self.mu_hat)
            if self.tc==True:
                rhat=np.maximum(-3.,np.minimum(3.,rhat))
            pi=np.exp(np.dot(np.array(Bs),theta))
            pi=pi/np.sum(pi)
            avg_rwd+=np.dot(rhat,pi)
#   
#            for bi in self.Bss[tau]:
#                rhat=np.dot(bi,self.mu_hat)
#                print(rhat)
#                if self.tc==True:
#                    rhat=np.maximum(-2.,np.minimum(2.,rhat))
#                pi=np.exp(np.dot(bi[self.d:],theta)) 
#                print(pi)
#                avg_rwd+=np.dot(rhat,pi/np.sum(pi))
                
        #reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,self.R).dot(theta)/self.t
        reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,theta)
        #print(reg_rwd)
        return -reg_rwd
          
    def update1(self,reward):
        self.rwds.append(reward)
        newb=np.array(list(self.Bs[self.action]))
        self.B=self.B+np.outer(newb,newb)
        self.y=self.y+reward*newb
        self.mu_hat=np.linalg.inv(self.B).dot(self.y)
        
    def update2(self):
        if self.t>9:
            if self.t%10==0:
                self.best=np.zeros(self.d)
                bestval=self.regularized_rwd(self.best)
                for m in range(50):
                    param=np.random.randint(-5,5,size=self.d)
                    paramval=self.regularized_rwd(param)
                    if paramval<bestval:
                        self.best=param
                        bestval=paramval
            
            self.theta=fmin(self.regularized_rwd, self.best)
            
    def j_theta_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=(-np.exp(np.dot(Bd,self.theta))+np.exp(2*np.dot(Bd,self.theta)))/(1+np.exp(np.dot(Bd,self.theta)))**3
        val=(np.dot(Bs[0],self.mu_hat)*temp-np.dot(Bs[1],self.mu_hat)*temp)*np.outer(Bd,Bd)-2*self.v*np.eye(self.d)
        return val
    
    def j_theta_mu(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.outer(Bd,Bs[0])-temp*np.outer(Bd,Bs[1])
        return val
    
    def expected_gram(self,Bs):
        pi=np.exp(np.dot(Bs,self.theta))
        pi=pi/np.sum(pi)
        val=np.outer(Bs[0],Bs[0])*pi[0]+np.outer(Bs[1],Bs[1])*pi[1]
        return val
    
    def j_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.dot(Bs[0],self.mu_hat)*Bd-temp*np.dot(Bs[1],self.mu_hat)*Bd-2*self.v*self.theta
        return val
    
    def variance(self):
        mean_j_theta_mu=np.zeros((self.d,self.d))
        mean_expected_gram=np.zeros((self.d,self.d))
        mean_j_theta_theta=np.zeros((self.d,self.d))
        mean_j_theta_outer=np.zeros((self.d,self.d))
        sigma_square=0
        for tau in range(self.t):
            Bs=self.Bss[tau]
            mean_j_theta_mu+=self.j_theta_mu(Bs)/self.t
            mean_expected_gram+=self.expected_gram(Bs)/self.t
            mean_j_theta_theta+=self.j_theta_theta(Bs)/self.t
            mean_j_theta_outer+=np.outer(self.j_theta(Bs),self.j_theta(Bs))/self.t
            sigma_square+=((self.rwds[tau]-np.dot(Bs[self.acs[tau]],self.mu_hat))**2)/self.t
        V_star=sigma_square*mean_j_theta_mu@np.linalg.inv(mean_expected_gram)@mean_j_theta_mu.T+mean_j_theta_outer
        Variance=np.linalg.inv(mean_j_theta_theta)@V_star@np.linalg.inv(mean_j_theta_theta)
        self.sigma_square=sigma_square
        return Variance/self.t
        

In [ ]:
## Actor Critic algorithm  With intercept !!!

#Algorithm for N=2

class ActorCritic_wintercept:
    def __init__(self,d,N,v,tc):
        self.mu_hat=np.zeros(d+1)
        self.y=np.zeros(d+1)
        self.B=np.eye(d+1)
        self.theta=np.zeros(d)
        self.v=v
        self.N=N
        self.d=d
        self.Bss=[]
        self.t=0
        self.tc=tc
        self.acs=[]
        self.rwds=[]
        #self.R=np.zeros((d,d))
        self.best=np.zeros(d)
    def select_ac(self,Bs):
        self.t=self.t+1
        self.Bs=Bs
        self.Bss.append(self.Bs)
        pi=np.array([np.exp(np.dot(bi,self.theta)) for bi in Bs])
        #if np.any(np.isinf(pi)):
        #    pi=list(np.isinf(pi)/np.sum(np.isinf(pi)))
        #else:
        pi=list(pi/np.sum(pi))
        #print(pi)
        self.action=np.random.choice(np.arange(self.N),1,p=pi)[0]
        #print(self.action)
        self.acs.append(self.action)
        
        #for bi in Bs:
        #    self.R+=np.outer(bi,bi)
        
        return(self.action)
    def regularized_rwd(self,theta):
        avg_rwd=0
        for tau in range(self.t):
            Bs=self.Bss[tau]
            Bs_1=np.hstack([np.ones(self.N).reshape(self.N,1),Bs])
            rhat=np.dot(Bs_1,self.mu_hat)
            if self.tc==True:
                rhat=np.maximum(-3.,np.minimum(3.,rhat))
            pi=np.exp(np.dot(np.array(Bs),theta))
            pi=pi/np.sum(pi)
            avg_rwd+=np.dot(rhat,pi)
#   
#            for bi in self.Bss[tau]:
#                rhat=np.dot(bi,self.mu_hat)
#                print(rhat)
#                if self.tc==True:
#                    rhat=np.maximum(-2.,np.minimum(2.,rhat))
#                pi=np.exp(np.dot(bi[self.d:],theta)) 
#                print(pi)
#                avg_rwd+=np.dot(rhat,pi/np.sum(pi))
                
        #reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,self.R).dot(theta)/self.t
        reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,theta)
        #print(reg_rwd)
        return -reg_rwd
          
    def update1(self,reward):
        self.rwds.append(reward)
        newb=np.array([1.]+list(self.Bs[self.action]))
        self.B=self.B+np.outer(newb,newb)
        self.y=self.y+reward*newb
        self.mu_hat=np.linalg.inv(self.B).dot(self.y)
        
    def update2(self):
        if self.t>9:
            if self.t%10==0:
                self.best=np.zeros(self.d)
                bestval=self.regularized_rwd(self.best)
                for m in range(50):
                    param=np.random.randint(-5,5,size=self.d)
                    paramval=self.regularized_rwd(param)
                    if paramval<bestval:
                        self.best=param
                        bestval=paramval
            
            self.theta=fmin(self.regularized_rwd, self.best)
            
    def j_theta_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=(-np.exp(np.dot(Bd,self.theta))+np.exp(2*np.dot(Bd,self.theta)))/(1+np.exp(np.dot(Bd,self.theta)))**3
        val=(np.dot(np.array([1.]+list(Bs[0])),self.mu_hat)*temp-np.dot(np.array([1.]+list(Bs[1])),self.mu_hat)*temp)*np.outer(Bd,Bd)-2*self.v*np.eye(self.d)
        return val
    
    def j_theta_mu(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.outer(Bd,np.array([1.]+list(Bs[0])))-temp*np.outer(Bd,np.array([1.]+list(Bs[1])))
        return val
    
    def expected_gram(self,Bs):
        pi=np.exp(np.dot(Bs,self.theta))
        pi=pi/np.sum(pi)
        Bs=np.hstack([np.ones(self.N).reshape(self.N,1),Bs])
        val=np.outer(Bs[0],Bs[0])*pi[0]+np.outer(Bs[1],Bs[1])*pi[1]
        return val
    
    def j_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.dot(np.array([1.]+list(Bs[0])),self.mu_hat)*Bd-temp*np.dot(np.array([1.]+list(Bs[1])),self.mu_hat)*Bd-2*self.v*self.theta
        return val
    
    def variance(self):
        mean_j_theta_mu=np.zeros((self.d,self.d+1))
        mean_expected_gram=np.zeros((self.d+1,self.d+1))
        mean_j_theta_theta=np.zeros((self.d,self.d))
        mean_j_theta_outer=np.zeros((self.d,self.d))
        sigma_square=0
        for tau in range(self.t):
            Bs=self.Bss[tau]
            mean_j_theta_mu+=self.j_theta_mu(Bs)/self.t
            mean_expected_gram+=self.expected_gram(Bs)/self.t
            mean_j_theta_theta+=self.j_theta_theta(Bs)/self.t
            mean_j_theta_outer+=np.outer(self.j_theta(Bs),self.j_theta(Bs))/self.t
            sigma_square+=((self.rwds[tau]-np.dot(np.array([1.]+list(Bs[self.acs[tau]])),self.mu_hat))**2)/self.t
        V_star=sigma_square*mean_j_theta_mu@np.linalg.inv(mean_expected_gram)@mean_j_theta_mu.T+mean_j_theta_outer
        Variance=np.linalg.inv(mean_j_theta_theta)@V_star@np.linalg.inv(mean_j_theta_theta)
        self.sigma_square=sigma_square
        return Variance/self.t
        

In [ ]:
## Proposed algorithm

#Algorithm for N=2

class Prop:
    def __init__(self,d,N,v,tc):
        self.mu_hat=np.zeros(d)
        self.theta=np.zeros(d)
        self.v=v
        self.N=N
        self.d=d
        self.Bss=[]
        self.t=0
        self.tc=tc
        self.acs=[]
        self.rwds=[]
        #self.R=np.zeros((d,d))
        self.best=np.zeros(d)
        self.thetas=[np.zeros(d)]
    def select_ac(self,Bs):
        self.t=self.t+1
        self.Bs=Bs
        self.Bss.append(self.Bs)
        pi=np.array([np.exp(np.dot(bi,self.theta)) for bi in Bs])
        #if np.any(np.isinf(pi)):
        #    pi=list(np.isinf(pi)/np.sum(np.isinf(pi)))
        #else:
        pi=list(pi/np.sum(pi))
        #print(pi)
        self.action=np.random.choice(np.arange(self.N),1,p=pi)[0]
        #print(self.action)
        self.acs.append(self.action)
        
        #for bi in Bs:
        #    self.R+=np.outer(bi,bi)
        
        return(self.action)
    
    def regularized_rwd(self,theta):
        avg_rwd=0
        for tau in range(self.t):
            Bs=self.Bss[tau]
            rhat=np.dot(Bs,self.mu_hat)
            if self.tc==True:
                rhat=np.maximum(-3.,np.minimum(3.,rhat))
            pi=np.exp(np.dot(Bs,theta))
            pi=pi/np.sum(pi)
            avg_rwd+=np.dot(rhat,pi)
#   
#            for bi in self.Bss[tau]:
#                rhat=np.dot(bi,self.mu_hat)
#                print(rhat)
#                if self.tc==True:
#                    rhat=np.maximum(-2.,np.minimum(2.,rhat))
#                pi=np.exp(np.dot(bi[self.d:],theta)) 
#                print(pi)
#                avg_rwd+=np.dot(rhat,pi/np.sum(pi))
                
        #reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,self.R).dot(theta)/self.t
        reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,theta)
        #print(reg_rwd)
        return -reg_rwd
          
    def update1(self,reward):
        self.rwds.append(reward)
        if self.t>9:
            B=np.zeros((self.d,self.d))
            y=np.zeros(self.d)
            for tau in range(self.t):
                Bs=self.Bss[tau]
                newb=np.array(list(Bs[self.acs[tau]]))
                pi=np.exp(np.dot(Bs,self.thetas[tau]))
                pi=pi/np.sum(pi)
                pi_recent=np.exp(np.dot(Bs,self.theta))
                pi_recent=pi_recent/np.sum(pi_recent)
                newb=newb-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1]
                B=B+np.outer(newb,newb)/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
                y=y+self.rwds[tau]*newb/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
            #print(newb,self.t, np.linalg.inv(B))
            self.mu_hat=np.linalg.inv(B).dot(y)
        
    def update2(self):
        if self.t>9:
            if self.t%10==0:
                self.best=np.zeros(self.d)
                bestval=self.regularized_rwd(self.best)
                for m in range(50):
                    param=np.random.randint(-5,5,size=self.d)
                    paramval=self.regularized_rwd(param)
                    if paramval<bestval:
                        self.best=param
                        bestval=paramval
            
            self.theta=fmin(self.regularized_rwd, self.best)
        self.thetas.append(self.theta)
            #print(self.theta)
            
    def j_theta_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=(-np.exp(np.dot(Bd,self.theta))+np.exp(2*np.dot(Bd,self.theta)))/(1+np.exp(np.dot(Bd,self.theta)))**3
        val=(np.dot(Bs[0],self.mu_hat)*temp-np.dot(Bs[1],self.mu_hat)*temp)*np.outer(Bd,Bd)-2*self.v*np.eye(self.d)
        return val
    
    def j_theta_mu(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.outer(Bd,Bs[0])-temp*np.outer(Bd,Bs[1])
        return val
    
    def j_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.dot(Bs[0],self.mu_hat)*Bd-temp*np.dot(Bs[1],self.mu_hat)*Bd-2*self.v*self.theta
        return val
    
    def u_mu(self,Bs,tau):
        pi=np.exp(np.dot(Bs,self.thetas[tau]))
        pi=pi/np.sum(pi)
        pi_recent=np.exp(np.dot(Bs,self.theta))
        pi_recent=pi_recent/np.sum(pi_recent)
        val=-2*(self.rwds[tau]-np.dot(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],self.mu_hat))*(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1])/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
        return val
    
    def u_mu_mu(self,Bs,tau):
        pi=np.exp(np.dot(Bs,self.thetas[tau]))
        pi=pi/np.sum(pi)
        pi_recent=np.exp(np.dot(Bs,self.theta))
        pi_recent=pi_recent/np.sum(pi_recent)        
        val=2*np.outer(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],
                       Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1])/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
        return val
    
    def u_mu_theta(self,Bs,tau):
        Bd=Bs[1]-Bs[0]
        pi=np.exp(np.dot(Bs,self.thetas[tau]))
        pi=pi/np.sum(pi)
        pi_recent=np.exp(np.dot(Bs,self.theta))
        pi_recent=pi_recent/np.sum(pi_recent)        
        temp=np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        temp=[-temp,temp]
        val_1=-2*np.outer(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],(np.dot(Bs[0],self.mu_hat)*(temp[0])+np.dot(Bs[1],self.mu_hat)*temp[1])*Bd)/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
        val_2=-2*(self.rwds[tau]-np.dot(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],self.mu_hat))*np.outer(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],temp[self.acs[tau]]*Bd)/pi[self.acs[tau]]
        val_3=2*(self.rwds[tau]-np.dot(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],self.mu_hat))/pi[self.acs[tau]]*pi_recent[self.acs[tau]]*temp[1]*np.outer(Bd,Bd)
        return val_1+val_2+val_3
    
    def variance(self):
        mean_j_theta_mu=np.zeros((self.d,self.d))
        mean_j_theta_theta=np.zeros((self.d,self.d))
        mean_j_theta_outer=np.zeros((self.d,self.d))
        
        mean_u_mu_theta=np.zeros((self.d,self.d))
        mean_u_mu_mu=np.zeros((self.d,self.d))
        mean_u_mu_outer=np.zeros((self.d,self.d))
        
        mean_u_mu_j_theta=np.zeros((self.d,self.d))
        
        for tau in range(self.t):
            Bs=self.Bss[tau]
            
            mean_j_theta_mu+=self.j_theta_mu(Bs)/self.t
            mean_j_theta_theta+=self.j_theta_theta(Bs)/self.t
            mean_j_theta_outer+=np.outer(self.j_theta(Bs),self.j_theta(Bs))/self.t
            
            mean_u_mu_theta+=self.u_mu_theta(Bs,tau)/self.t
            mean_u_mu_mu+=self.u_mu_mu(Bs,tau)/self.t
            mean_u_mu_outer+=np.outer(self.u_mu(Bs,tau),self.u_mu(Bs,tau))/self.t
            
            mean_u_mu_j_theta+=np.outer(self.u_mu(Bs,tau),self.j_theta(Bs))/self.t
        
        Bread=np.zeros((self.d*(2),self.d*(2)))
        Bread[:(self.d),:(self.d)]=mean_u_mu_mu
        Bread[:(self.d),(self.d):]=mean_u_mu_theta
        Bread[(self.d):,:(self.d)]=mean_j_theta_mu
        Bread[(self.d):,(self.d):]=mean_j_theta_theta
        Meat=np.zeros((self.d*(2),self.d*(2)))
        Meat[:(self.d),:(self.d)]=mean_u_mu_outer
        Meat[:(self.d),(self.d):]=mean_u_mu_j_theta
        Meat[(self.d):,:(self.d)]=mean_u_mu_j_theta.T
        Meat[(self.d):,(self.d):]=mean_j_theta_outer
        Meat=Meat/self.t
        Variance=np.linalg.inv(Bread)@Meat@np.linalg.inv(Bread.T)
        Variance=Variance[(self.d):,(self.d):]
        return Variance
        

## Fast version

In [ ]:
## Actor Critic algorithm
#  rejection sampling 을 해야 하는 경우가 아니면 기존 알고리즘을 써도 상관 없음
# 얘도 intercept 있음

#Algorithm for N=2 

class ActorCritic_fast:
    def __init__(self,d,N,v,tc):
        self.mu_hat=np.zeros(d+1)
        self.y=np.zeros(d+1)
        self.B=np.eye(d+1)
        self.theta=np.zeros(d)
        self.v=v
        self.N=N
        self.d=d
        self.Bss=[]
        self.t=0
        self.tc=tc
        self.acs=[]
        self.rwds=[]
        #self.R=np.zeros((d,d))
        self.best=np.zeros(d)
    def select_ac(self,Bs):
        #self.t=self.t+1
        self.Bs=Bs
        #self.Bss.append(self.Bs)
        pi=np.array([np.exp(np.dot(bi,self.theta)) for bi in Bs])
        #if np.any(np.isinf(pi)):
        #    pi=list(np.isinf(pi)/np.sum(np.isinf(pi)))
        #else:
        pi=list(pi/np.sum(pi))
        self.pi=pi
        #print(pi)
        self.action=np.random.choice(np.arange(self.N),1,p=pi)[0]
        #print(self.action)
        #self.acs.append(self.action)
        
        #for bi in Bs:
        #    self.R+=np.outer(bi,bi)
        
        return(self.action)
    def regularized_rwd(self,theta):
        avg_rwd=0
        for tau in range(self.t):
            Bs=self.Bss[tau]
            Bs_1=np.hstack([np.ones(self.N).reshape(self.N,1),Bs])
            rhat=np.dot(Bs_1,self.mu_hat)
            if self.tc==True:
                rhat=np.maximum(-3.,np.minimum(3.,rhat))
            pi=np.exp(np.dot(np.array(Bs),theta))
            pi=pi/np.sum(pi)
            avg_rwd+=np.dot(rhat,pi)
#   
#            for bi in self.Bss[tau]:
#                rhat=np.dot(bi,self.mu_hat)
#                print(rhat)
#                if self.tc==True:
#                    rhat=np.maximum(-2.,np.minimum(2.,rhat))
#                pi=np.exp(np.dot(bi[self.d:],theta)) 
#                print(pi)
#                avg_rwd+=np.dot(rhat,pi/np.sum(pi))
                
        #reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,self.R).dot(theta)/self.t
        reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,theta)
        #print(reg_rwd)
        return -reg_rwd
          
    def update1(self,reward):
        self.t=self.t+1
        self.Bss.append(self.Bs)
        self.acs.append(self.action)
        self.rwds.append(reward)
        newb=np.array([1.]+list(self.Bs[self.action]))
        self.B=self.B+np.outer(newb,newb)
        self.y=self.y+reward*newb
        self.mu_hat=np.linalg.inv(self.B).dot(self.y)
        
    def update2(self):
        if self.t>9:
            if self.t%10==0:
                self.best=np.zeros(self.d)
                bestval=self.regularized_rwd(self.best)
                for m in range(50):
                    param=np.random.randint(-5,5,size=self.d)
                    paramval=self.regularized_rwd(param)
                    if paramval<bestval:
                        self.best=param
                        bestval=paramval
            
            self.theta=fmin(self.regularized_rwd, self.best)
            
    def j_theta_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=(-np.exp(np.dot(Bd,self.theta))+np.exp(2*np.dot(Bd,self.theta)))/(1+np.exp(np.dot(Bd,self.theta)))**3
        val=(np.dot(np.array([1.]+list(Bs[0])),self.mu_hat)*temp-np.dot(np.array([1.]+list(Bs[1])),self.mu_hat)*temp)*np.outer(Bd,Bd)-2*self.v*np.eye(self.d)
        return val
    
    def j_theta_mu(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.outer(Bd,np.array([1.]+list(Bs[0])))-temp*np.outer(Bd,np.array([1.]+list(Bs[1])))
        return val
    
    def expected_gram(self,Bs):
        pi=np.exp(np.dot(Bs,self.theta))
        pi=pi/np.sum(pi)
        Bs=np.hstack([np.ones(self.N).reshape(self.N,1),Bs])
        val=np.outer(Bs[0],Bs[0])*pi[0]+np.outer(Bs[1],Bs[1])*pi[1]
        return val
    
    def j_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.dot(np.array([1.]+list(Bs[0])),self.mu_hat)*Bd-temp*np.dot(np.array([1.]+list(Bs[1])),self.mu_hat)*Bd-2*self.v*self.theta
        return val
    
    def variance(self):
        mean_j_theta_mu=np.zeros((self.d,self.d+1))
        mean_expected_gram=np.zeros((self.d+1,self.d+1))
        mean_j_theta_theta=np.zeros((self.d,self.d))
        mean_j_theta_outer=np.zeros((self.d,self.d))
        sigma_square=0
        for tau in range(self.t):
            Bs=self.Bss[tau]
            mean_j_theta_mu+=self.j_theta_mu(Bs)/self.t
            mean_expected_gram+=self.expected_gram(Bs)/self.t
            mean_j_theta_theta+=self.j_theta_theta(Bs)/self.t
            mean_j_theta_outer+=np.outer(self.j_theta(Bs),self.j_theta(Bs))/self.t
            sigma_square+=((self.rwds[tau]-np.dot(np.array([1.]+list(Bs[self.acs[tau]])),self.mu_hat))**2)/self.t
        V_star=sigma_square*mean_j_theta_mu@np.linalg.pinv(mean_expected_gram)@mean_j_theta_mu.T+mean_j_theta_outer
        Variance=np.linalg.inv(mean_j_theta_theta)@V_star@np.linalg.inv(mean_j_theta_theta)
        self.sigma_square=sigma_square
        return Variance/self.t
        

In [ ]:
## Proposed algorithm
# 기존 알고리즘과 다른 부분은 B matrix 를 0 matrix 로 시작하지 않고 0.01*I 로 시작함.

#Algorithm for N=2

class Prop_fast:
    def __init__(self,d,N,v,tc,stepsize):
        self.mu_hat=np.zeros(d)
        self.theta=np.zeros(d)
        self.v=v
        self.N=N
        self.d=d
        self.Bss=[]
        self.t=0
        self.tc=tc
        self.acs=[]
        self.rwds=[]
        #self.R=np.zeros((d,d))
        self.best=np.zeros(d)
        self.thetas=[np.zeros(d)  for i in range(stepsize)]
        self.stepsize=stepsize
    def select_ac(self,Bs):
        #self.t=self.t+1
        self.Bs=Bs
        #self.Bss.append(self.Bs)
        pi=np.array([np.exp(np.dot(bi,self.theta)) for bi in Bs])
        #if np.any(np.isinf(pi)):
        #    pi=list(np.isinf(pi)/np.sum(np.isinf(pi)))
        #else:
        pi=list(pi/np.sum(pi))
        self.pi=pi
        #print(pi)
        self.action=np.random.choice(np.arange(self.N),1,p=pi)[0]
        #print(self.action)
        #self.acs.append(self.action)
        
        #for bi in Bs:
        #    self.R+=np.outer(bi,bi)
        
        return(self.action)
    
    def regularized_rwd(self,theta):
        avg_rwd=0
        for tau in range(self.t):
            Bs=self.Bss[tau]
            rhat=np.dot(Bs,self.mu_hat)
            if self.tc==True:
                rhat=np.maximum(-3.,np.minimum(3.,rhat))
            pi=np.exp(np.dot(Bs,theta))
            pi=pi/np.sum(pi)
            avg_rwd+=np.dot(rhat,pi)
#   
#            for bi in self.Bss[tau]:
#                rhat=np.dot(bi,self.mu_hat)
#                print(rhat)
#                if self.tc==True:
#                    rhat=np.maximum(-2.,np.minimum(2.,rhat))
#                pi=np.exp(np.dot(bi[self.d:],theta)) 
#                print(pi)
#                avg_rwd+=np.dot(rhat,pi/np.sum(pi))
                
        #reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,self.R).dot(theta)/self.t
        reg_rwd=avg_rwd/self.t-self.v*np.dot(theta,theta)
        #print(reg_rwd)
        return -reg_rwd
          
    def update1_1(self,reward):
        self.t=self.t+1
        self.Bss.append(self.Bs)
        self.acs.append(self.action)
        self.rwds.append(reward)
    
    def update1_2(self):
        if self.t>9:
            B=0.01*np.eye(self.d)
            Binv=100*np.eye(self.d)
            y=np.zeros(self.d)
            for tau in range(self.t):
                Bs=self.Bss[tau]
                newb=np.array(list(Bs[self.acs[tau]]))
                pi=np.exp(np.dot(Bs,self.thetas[tau]))
                pi=pi/np.sum(pi)
                pi_recent=np.exp(np.dot(Bs,self.theta))
                pi_recent=pi_recent/np.sum(pi_recent)
                newb=newb-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1]
                B=B+np.outer(newb,newb)/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
                y=y+self.rwds[tau]*newb/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
                Binv=Binv-(pi_recent[self.acs[tau]]/pi[self.acs[tau]])*((Binv).dot(np.outer(newb,newb))).dot(Binv)/(1.+np.dot(newb,(Binv).dot(newb))*(pi_recent[self.acs[tau]]/pi[self.acs[tau]]))
            #print(newb,self.t, np.linalg.inv(B))
            self.mu_hat=Binv.dot(y)
        
    def update2(self):
        if self.t>9:
            if self.t%10==0:
                self.best=np.zeros(self.d)
                bestval=self.regularized_rwd(self.best)
                for m in range(50):
                    param=np.random.randint(-5,5,size=self.d)
                    paramval=self.regularized_rwd(param)
                    if paramval<bestval:
                        self.best=param
                        bestval=paramval
            
            self.theta=fmin(self.regularized_rwd, self.best)
        #self.thetas.append(self.theta)
        self.thetas=self.thetas+[self.theta for i in range(self.stepsize)]
            #print(self.theta)
            
    def j_theta_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=(-np.exp(np.dot(Bd,self.theta))+np.exp(2*np.dot(Bd,self.theta)))/(1+np.exp(np.dot(Bd,self.theta)))**3
        val=(np.dot(Bs[0],self.mu_hat)*temp-np.dot(Bs[1],self.mu_hat)*temp)*np.outer(Bd,Bd)-2*self.v*np.eye(self.d)
        return val
    
    def j_theta_mu(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.outer(Bd,Bs[0])-temp*np.outer(Bd,Bs[1])
        return val
    
    def j_theta(self,Bs):
        Bd=Bs[1]-Bs[0]
        temp=-np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        val=temp*np.dot(Bs[0],self.mu_hat)*Bd-temp*np.dot(Bs[1],self.mu_hat)*Bd-2*self.v*self.theta
        return val
    
    def u_mu(self,Bs,tau):
        pi=np.exp(np.dot(Bs,self.thetas[tau]))
        pi=pi/np.sum(pi)
        pi_recent=np.exp(np.dot(Bs,self.theta))
        pi_recent=pi_recent/np.sum(pi_recent)
        val=-2*(self.rwds[tau]-np.dot(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],self.mu_hat))*(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1])/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
        return val
    
    def u_mu_mu(self,Bs,tau):
        pi=np.exp(np.dot(Bs,self.thetas[tau]))
        pi=pi/np.sum(pi)
        pi_recent=np.exp(np.dot(Bs,self.theta))
        pi_recent=pi_recent/np.sum(pi_recent)        
        val=2*np.outer(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],
                       Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1])/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
        return val
    
    def u_mu_theta(self,Bs,tau):
        Bd=Bs[1]-Bs[0]
        pi=np.exp(np.dot(Bs,self.thetas[tau]))
        pi=pi/np.sum(pi)
        pi_recent=np.exp(np.dot(Bs,self.theta))
        pi_recent=pi_recent/np.sum(pi_recent)        
        temp=np.exp(np.dot(Bd,self.theta))/(1+np.exp(np.dot(Bd,self.theta)))**2
        temp=[-temp,temp]
        val_1=-2*np.outer(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],(np.dot(Bs[0],self.mu_hat)*(temp[0])+np.dot(Bs[1],self.mu_hat)*temp[1])*Bd)/pi[self.acs[tau]]*pi_recent[self.acs[tau]]
        val_2=-2*(self.rwds[tau]-np.dot(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],self.mu_hat))*np.outer(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],temp[self.acs[tau]]*Bd)/pi[self.acs[tau]]
        val_3=2*(self.rwds[tau]-np.dot(Bs[self.acs[tau]]-Bs[0]*pi_recent[0]-Bs[1]*pi_recent[1],self.mu_hat))/pi[self.acs[tau]]*pi_recent[self.acs[tau]]*temp[1]*np.outer(Bd,Bd)
        return val_1+val_2+val_3
    
    def variance(self):
        mean_j_theta_mu=np.zeros((self.d,self.d))
        mean_j_theta_theta=np.zeros((self.d,self.d))
        mean_j_theta_outer=np.zeros((self.d,self.d))
        
        mean_u_mu_theta=np.zeros((self.d,self.d))
        mean_u_mu_mu=np.zeros((self.d,self.d))
        mean_u_mu_outer=np.zeros((self.d,self.d))
        
        mean_u_mu_j_theta=np.zeros((self.d,self.d))
        
        for tau in range(self.t):
            Bs=self.Bss[tau]
            
            mean_j_theta_mu+=self.j_theta_mu(Bs)/self.t
            mean_j_theta_theta+=self.j_theta_theta(Bs)/self.t
            mean_j_theta_outer+=np.outer(self.j_theta(Bs),self.j_theta(Bs))/self.t
            
            mean_u_mu_theta+=self.u_mu_theta(Bs,tau)/self.t
            mean_u_mu_mu+=self.u_mu_mu(Bs,tau)/self.t
            mean_u_mu_outer+=np.outer(self.u_mu(Bs,tau),self.u_mu(Bs,tau))/self.t
            
            mean_u_mu_j_theta+=np.outer(self.u_mu(Bs,tau),self.j_theta(Bs))/self.t
        
        Bread=np.zeros((self.d*(2),self.d*(2)))
        Bread[:(self.d),:(self.d)]=mean_u_mu_mu
        Bread[:(self.d),(self.d):]=mean_u_mu_theta
        Bread[(self.d):,:(self.d)]=mean_j_theta_mu
        Bread[(self.d):,(self.d):]=mean_j_theta_theta
        Meat=np.zeros((self.d*(2),self.d*(2)))
        Meat[:(self.d),:(self.d)]=mean_u_mu_outer
        Meat[:(self.d),(self.d):]=mean_u_mu_j_theta
        Meat[(self.d):,:(self.d)]=mean_u_mu_j_theta.T
        Meat[(self.d):,(self.d):]=mean_j_theta_outer
        Meat=Meat/self.t
        Variance=np.linalg.pinv(Bread)@Meat@np.linalg.pinv(Bread.T)
        Variance=Variance[(self.d):,(self.d):]
        return Variance
        

### Epsilon greedy

In [ ]:
# with intercept

class Epsilon_greedy:
    def __init__(self,d,N,epsilon):
        self.mu_hat=[np.zeros(d+1),np.zeros(d+1)]
        self.B=[0.001*np.eye(d+1), 0.001*np.eye(d+1)]
        self.B_inv=[1000*np.eye(d+1), 1000*np.eye(d+1)]
        self.y=[np.zeros(d+1), np.zeros(d+1)]
        self.epsilon=epsilon
        self.N=N
        self.d=d
        self.Sigma=np.zeros((d+1,d+1))
        #self.Sigma_inv=np.zeros((d+1,d+1))
        self.t=0
        self.pis=[]
        self.acs=[]
        self.Bss=[]
        self.rwds=[]
        
    def select_ac(self,b):
        self.b=np.array([1.]+list(b))
        pi=(1-self.epsilon)*int(self.b.dot(self.mu_hat[1])>self.b.dot(self.mu_hat[0]))+self.epsilon/2.
        self.pi=[1-pi,pi]
        self.action=np.random.choice(np.arange(self.N),1,p=self.pi)[0]
        
        return self.action
    
    def update(self,reward):
        self.t+=1
        self.pis.append(self.pi)
        self.acs.append(self.action)
        self.Bss.append(self.b)
                
        self.B[self.action]+=np.outer(self.b, self.b)/self.pi[self.action]
        self.Sigma+=np.outer(self.b, self.b)
        
        #if self.t==50:
            #self.B_inv[0]=np.linalg.inv(self.B[0])
            #self.B_inv[1]=np.linalg.inv(self.B[1])
            
        #    self.Sigma_inv=np.linalg.inv(self.Sigma)
        
        #if self.t>50:

        #    temp=self.Sigma_inv
        #    self.Sigma_inv=temp-temp.dot(np.outer(self.b, self.b)).dot(temp)/(1+self.b.dot(temp).dot(self.b))
            
        temp=self.B_inv[self.action]
        self.B_inv[self.action]=temp-temp.dot(np.outer(self.b, self.b)/self.pi[self.action]).dot(temp)/(1+self.b.dot(temp).dot(self.b)/self.pi[self.action])
       
        self.y[self.action]+=self.b*reward/self.pi[self.action]
        
        self.mu_hat[self.action]=self.B_inv[self.action].dot(self.y[self.action])
    
        self.rwds.append(reward)
        
    def variance(self):
        self.Sigma_inv=np.linalg.inv(self.Sigma)*self.t
        H=[np.zeros((self.d+1,self.d+1)), np.zeros((self.d+1,self.d+1))]
        for tau in range(self.t):
            var=(self.rwds[tau]-self.Bss[tau].dot(self.mu_hat[self.acs[tau]]))**2
            H[self.acs[tau]]+=var*np.outer(self.Bss[tau], self.Bss[tau])/(self.pis[tau][self.acs[tau]]**2)
        H[0]/=self.t
        H[1]/=self.t
        variance_0=self.Sigma_inv.dot(H[0]).dot(self.Sigma_inv)/self.t
        variance_1=self.Sigma_inv.dot(H[1]).dot(self.Sigma_inv)/self.t
        return (variance_0, variance_1)
        
        